In [8]:
# Data Akun
akun_email = "email@gmail.com"
akun_pwd   = "Password1234"

# Data Penumpang
pnp_nama = "Nama Lengkap"
pnp_gender = "MR"    # Tuan=MR; Nyonya=MRS.  (tanda titik jangan ketinggalan)
pnp_NIK   = "330687912229404"

# Data Kereta Api
sta_asal    = "KAC"
sta_tujuan  = "GB"
tanggal     = "2023-04-25"
dewasa_jml  = "1"
bayi_jml    = "0"
nama_ka     = "lodaya"  # lowercase

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
from datetime import datetime
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import threading

# Set sound alert
import winsound
duration = 750  # milliseconds
freq = 1000  # Hz

# Membuat objek browser webdriver
driver = webdriver.Edge(executable_path="C:\webdriver\edgedriver_win64\msedgedriver.exe")

def login(email, password):
    url_login = "https://booking.kai.id/auth/login" 
    
    # Membuka halaman web
    driver.get(url_login)
    
    # Locating element
    box_username = driver.find_element(By.XPATH, "/html/body/div/div/div/div/div/div/div/div[2]/form/div[1]/div/div[1]/input")
    box_password = driver.find_element(By.XPATH, "/html/body/div/div/div/div/div/div/div/div[2]/form/div[2]/div/div[1]/input")
    
    but_login = driver.find_element(By.XPATH, "/html/body/div/div/div/div/div/div/div/div[2]/form/div[3]/div/button")
    
    # Input data
    box_username.send_keys(email)
    box_password.send_keys(password)
    
    but_login.click()
    
def set_penumpang(gender, nama, nik):
    # locating element
    box_gender = driver.find_element(By.ID, "penumpang_title1")
    select_gender = Select(box_gender)
    box_nama   = driver.find_element(By.ID, "penumpang_nama1")
    box_nik   = driver.find_element(By.ID, "penumpang_notandapengenal1")
    
    select_gender.select_by_value(gender) # MR atau MRS.
    box_nama.send_keys(nama)
    box_nik.send_keys(nik)
    
    
def beli_tiket( pnp_nama, pnp_gender, pnp_NIK, sta_asal, sta_tujuan, tanggal, dewasa_jml, bayi_jml, nama_ka):
    link = "https://booking.kai.id/" 

    # Membuka halaman web
    driver.get(link)

    current_dateTime = datetime.now()
    bulan_now = str(current_dateTime)[5:7]
    bulan_buy   = tanggal[5:7]

    # Locating xpath elemen sta_asal, sta_tujuan, dan tanggal
    box_sta_asal = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div[2]/div[2]/form/div[1]/div[1]/div/input[2]")
    box_sta_tujuan = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div[2]/div[2]/form/div[1]/div[2]/div/input[2]")
    box_date_click = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div[2]/div[2]/form/div[2]/div[1]/div[1]/input")

    # Input sta_asal
    box_sta_asal.send_keys(sta_asal)
    box_sta_asal = None
    while not box_sta_asal:
        try:
            box_sta_asal = driver.find_element(By.XPATH, "/html/body/ul/li[2]/span[1]")
            box_sta_asal.click()
        except NoSuchElementException:
            time.sleep(1)

    # Input sta_tujuan
    box_sta_tujuan.send_keys(sta_tujuan)
    box_sta_tujuan = None
    while not box_sta_tujuan:
        try:
            box_sta_tujuan = driver.find_element(By.XPATH, "/html/body/ul/li[2]/span[1]")
            box_sta_tujuan.click()
        except NoSuchElementException:
            time.sleep(1)

    # Input tanggal
    box_date_click.click()
    box_month = driver.find_element(By.CLASS_NAME, "ui-datepicker-month")
    select = Select(box_month)
    select.select_by_value(str(int(bulan_buy)-int(bulan_now)+2))

    # Menemukan semua element <td>
    td_elements = driver.find_elements(By.CLASS_NAME, "ui-state-default")

    hari_buy = tanggal[-2:]
    # Mencari element <td> yang berisi tanggal yang ditentukan
    for td in td_elements:
        # print(td.text)
        if td.text == hari_buy:
            td.click()

    # Locating xpath elemen jml dewasa dan bayi
    box_dewasa_jml = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div[2]/div[2]/form/div[2]/div[2]/div/input")
    box_bayi_jml = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div[2]/div[2]/form/div[2]/div[3]/div/input")

    # Membersihkan isi textbox
    box_dewasa_jml.send_keys(Keys.BACK_SPACE)
    box_bayi_jml.send_keys(Keys.BACK_SPACE)

    box_dewasa_jml.send_keys(dewasa_jml)
    box_bayi_jml.send_keys(bayi_jml)

    # Search tiket
    but_search = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div[2]/div[2]/form/div[3]/div/input")
    but_search.click()

    # Mencari dan memilih kereta target
    kereta_tgt = None
    while not kereta_tgt:
        try:
            kereta_tgt = driver.find_element(By.XPATH, "//div[contains(@class, 'data-block list-kereta') and @data-kereta='{}']".format(nama_ka)  + "//form[@method='POST']//a[@class='card-schedule']")
            if kereta_tgt:
                kereta_tgt.click()
                print("DAPAT kereta {}!!!".format(nama_ka))


        except NoSuchElementException:
            print("mencari kereta {}...".format(nama_ka))
            driver.refresh()

            time.sleep(3)

    # Pesan tiket
    time.sleep(3)

    # INPUT DATA PENUMPANG
    set_penumpang(pnp_gender, pnp_nama, pnp_NIK)

    # centang setuju
    box_setuju = driver.find_element(By.ID, "setuju")
    box_setuju.click()


    for i in range(0, 100):
        winsound.Beep(freq, duration)

if __name__ =="__main__":
    # login akun
    login(akun_email, akun_pwd)
    time.sleep(3)
    
    # War Ticket!
    beli_tiket(pnp_nama, pnp_gender, pnp_NIK, sta_asal, sta_tujuan, tanggal, dewasa_jml, bayi_jml, nama_ka)
    print("Done!")

C:\Users\defit\AppData\Local\Temp\ipykernel_1976\627630598.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge(executable_path="C:\webdriver\edgedriver_win64\msedgedriver.exe")


DAPAT kereta lodaya!!!


KeyboardInterrupt: 